In [1]:
liked_books = ["30624788", "27071490", "6186357", "19486412", "18747396", "12182809", "271199"]

In [2]:
!head book_id_map.csv

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [3]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [4]:
len(csv_book_mapping)

2360651

In [5]:
!wc -l goodreads_interactions.csv

 228648343 goodreads_interactions.csv


In [6]:
!ls -lh | grep goodreads_interactions.csv

-rw-r--r--  1 rach  staff   4.0G Apr 29 00:41 goodreads_interactions.csv


In [7]:
!head goodreads_interactions.csv

user_id,book_id,is_read,rating,is_reviewed
0,948,1,5,0
0,947,1,5,1
0,946,1,5,0
0,945,1,5,0
0,944,1,5,0
0,943,1,5,0
0,942,1,5,0
0,941,1,5,0
0,940,1,5,0


In [8]:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue
            
        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
            overlap_users.add(user_id)

Recommendation list that only contains books that users who have liked the same books as me have read

In [9]:
rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
    
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [10]:
len(overlap_users)

70728

In [11]:
len(rec_lines)

37695679

In [12]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [13]:
top_recs = recs["book_id"].value_counts().head(10)
top_recs = top_recs.index.values

In [14]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [15]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook


In [16]:
books_titles[books_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,mod_title
73489,7260188,"Mockingjay (The Hunger Games, #3)",1743362,https://www.goodreads.com/book/show/7260188-mo...,https://images.gr-assets.com/books/1358275419m...,mockingjay the hunger games 3
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
323031,41865,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
373254,19063,The Book Thief,1193697,https://www.goodreads.com/book/show/19063.The_...,https://images.gr-assets.com/books/1390053681m...,the book thief
524391,6186357,"The Maze Runner (Maze Runner, #1)",745820,https://www.goodreads.com/book/show/6186357-th...,https://images.gr-assets.com/books/1375596592m...,the maze runner maze runner 1
790927,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
833311,6148028,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,catching fire the hunger games 2
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
1095301,11870085,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,the fault in our stars
1270797,13335037,"Divergent (Divergent, #1)",1962813,https://www.goodreads.com/book/show/13335037-d...,https://images.gr-assets.com/books/1328559506m...,divergent divergent 1


In [17]:
all_recs = recs["book_id"].value_counts()

In [18]:
all_recs

2767052     53810
3           45179
6186357     44964
11870085    43248
6148028     40332
            ...  
17346184        1
1790283         1
21295715        1
13617055        1
6952233         1
Name: book_id, Length: 1288606, dtype: int64

In [19]:
all_recs = all_recs.to_frame().reset_index()

In [20]:
all_recs

,index,book_id
0,2767052,53810
1,3,45179
2,6186357,44964
3,11870085,43248
4,6148028,40332
...,...,...
1288601,17346184,1
1288602,1790283,1
1288603,21295715,1
1288604,13617055,1


In [21]:
all_recs.columns = ["book_id", "book_count"]

In [22]:
all_recs

,book_id,book_count
0,2767052,53810
1,3,45179
2,6186357,44964
3,11870085,43248
4,6148028,40332
...,...,...
1288601,17346184,1
1288602,1790283,1
1288603,21295715,1
1288604,13617055,1


In [23]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")

In [24]:
all_recs

,book_id,book_count,title,ratings,url,cover_image,mod_title
0,2767052,53810,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
1,3,45179,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
2,6186357,44964,"The Maze Runner (Maze Runner, #1)",745820,https://www.goodreads.com/book/show/6186357-th...,https://images.gr-assets.com/books/1375596592m...,the maze runner maze runner 1
3,11870085,43248,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,the fault in our stars
4,6148028,40332,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,catching fire the hunger games 2
...,...,...,...,...,...,...,...
1018976,20403841,1,Automne,17,https://www.goodreads.com/book/show/20403841-a...,https://images.gr-assets.com/books/1388502395m...,automne
1018977,215671,1,Ebony Angel,101,https://www.goodreads.com/book/show/215671.Ebo...,https://s.gr-assets.com/assets/nophoto/book/11...,ebony angel
1018978,17346184,1,"Red Velvet Romance, The Art of Seduction",25,https://www.goodreads.com/book/show/17346184-r...,https://images.gr-assets.com/books/1360525290m...,red velvet romance the art of seduction
1018979,13617055,1,"For the Love of Chocolat (Lee's Girls, #2)",156,https://www.goodreads.com/book/show/13617055-f...,https://images.gr-assets.com/books/1342385220m...,for the love of chocolat lees girls 2


In [25]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])

In [26]:
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
2561,24909347,1803,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,39644.012195
6127,26856502,835,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,19920.714286
4242,29749098,1163,"Catwoman: Soulstealer (DC Icons, #3)",73,https://www.goodreads.com/book/show/29749098-c...,https://s.gr-assets.com/assets/nophoto/book/11...,catwoman soulstealer dc icons 3,18528.342466
8596,31520883,607,"Untitled (An Ember in the Ashes, #4)",22,https://www.goodreads.com/book/show/31520883-u...,https://s.gr-assets.com/assets/nophoto/book/11...,untitled an ember in the ashes 4,16747.681818
3358,30809786,1431,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,https://www.goodreads.com/book/show/30809786-a...,https://images.gr-assets.com/books/1507476834m...,a reaper at the gates an ember in the ashes 3,16514.201613
2862,17699859,1636,"Chain of Thorns (The Last Hours, #3)",185,https://www.goodreads.com/book/show/17699859-c...,https://s.gr-assets.com/assets/nophoto/book/11...,chain of thorns the last hours 3,14467.545946
1543,13541056,2730,The Queen of Air and Darkness (The Dark Artifi...,516,https://www.goodreads.com/book/show/13541056-q...,https://images.gr-assets.com/books/1510447136m...,the queen of air and darkness the dark artific...,14443.604651
2312,17699853,1983,"Chain of Gold (The Last Hours, #1)",287,https://www.goodreads.com/book/show/17699853-c...,https://s.gr-assets.com/assets/nophoto/book/11...,chain of gold the last hours 1,13701.355401
9699,36307629,542,"King of Scars (King of Scars, #1)",22,https://www.goodreads.com/book/show/36307629-k...,https://images.gr-assets.com/books/1506962795m...,king of scars king of scars 1,13352.909091
5027,33645809,1002,"Legendary (Caraval, #2)",76,https://www.goodreads.com/book/show/33645809-l...,https://images.gr-assets.com/books/1506803078m...,legendary caraval 2,13210.578947


In [27]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)

In [28]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<img src="{}" width=50></img>'.format(val, val)

In [29]:
popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
2561,24909347,1803,"Obsidio (The Illuminae Files, #3)",82,Goodreads,,obsidio the illuminae files 3,39644.012195
6127,26856502,835,"Vengeful (Villains, #2)",35,Goodreads,,vengeful villains 2,19920.714286
4242,29749098,1163,"Catwoman: Soulstealer (DC Icons, #3)",73,Goodreads,,catwoman soulstealer dc icons 3,18528.342466
8596,31520883,607,"Untitled (An Ember in the Ashes, #4)",22,Goodreads,,untitled an ember in the ashes 4,16747.681818
3358,30809786,1431,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,Goodreads,,a reaper at the gates an ember in the ashes 3,16514.201613
2862,17699859,1636,"Chain of Thorns (The Last Hours, #3)",185,Goodreads,,chain of thorns the last hours 3,14467.545946
1543,13541056,2730,"The Queen of Air and Darkness (The Dark Artifices, #3)",516,Goodreads,,the queen of air and darkness the dark artifices 3,14443.604651
2312,17699853,1983,"Chain of Gold (The Last Hours, #1)",287,Goodreads,,chain of gold the last hours 1,13701.355401
9699,36307629,542,"King of Scars (King of Scars, #1)",22,Goodreads,,king of scars king of scars 1,13352.909091
5027,33645809,1002,"Legendary (Caraval, #2)",76,Goodreads,,legendary caraval 2,13210.578947
